In [1]:
using LinearAlgebra
using Distributions
using Optim
using Random
using StatsFuns
using JuMP
using MosekTools
using StatsBase
using SparseArrays # 可选，用于处理稀疏性（如果数据量很大）
using FileIO
using JLD2
using Plots
using LaTeXStrings

using DataFrames, Colors
using StatsPlots   # 提供 boxplot，基于 Plots

include("Data_Generation_PLD.jl")
# include("Estimation_PLD.jl")
# include("Models_PLD.jl")
# include("Evaluation_PLD.jl")

Generate_Wang_Qi_Max_True_Data (generic function with 1 method)

In [2]:
N = 2 # num of product
N_x = 20 # num of product feature
c_l = ones(N_x)  # X * c_l >= d_r
d_r = ones(N) * 5
rev_gap = 0.001
N_u = 5 # num of customer feature
S_train = 200 # num of training samples
S_test = 1 # num of training samples
m = 5 # num of candidates in training samples
N_nonzero = 5 # num of nonzero entries in A
lambda_list = [0.01]
gamma_list = [0.0,0.01,0.05,0.1,0.15,0.2,0.3,0.4,0.5]
gamma_list = [0.0]

instances = 10
Random.seed!(2)
project_dir = "N=$(N)_N_x=$(N_x)_N_u=$(N_u)_S_train=$(S_train)_N_nonzero=$(N_nonzero)/"

"N=2_N_x=20_N_u=5_S_train=200_N_nonzero=5/"

In [3]:
current_dir = pwd()
parent_dir = dirname(current_dir)
grand_pa_dir = dirname(parent_dir)
data_dir = string(dirname(grand_pa_dir), "/Data/Product_Line_Design/")
if !isdir(data_dir)
    mkpath(data_dir)
end
data_dir = string(data_dir,project_dir)
println("Data directory: ", data_dir)

Data directory: /Users/zhangxun/Dropbox/Research/Robust_Exp/Data/Product_Line_Design/N=2_N_x=20_N_u=5_S_train=200_N_nonzero=5/


#### Data Generation

In [4]:
function compute_w(alpha0,alpha,beta,A,z_input)
    nu0 = alpha0 + beta' * z_input;
    nu = alpha .+ A * z_input;
    return nu0,nu
end

compute_w (generic function with 1 method)

In [5]:
Input_Data = Dict()
ins = 1
while ins <= instances
    data_dir_ins = string(data_dir, "instance=$ins/")
    # ****** Data ******
    theta_true, r_params = Generate_Wang_Qi_Max_True_Paras(N_x,N_u,N_nonzero);
    X_train,Y_train,Z_train = Generate_Wang_Qi_Max_True_Data(N_x, N_u, S_train, m,theta_true);
    X_test,Y_test,Z_test = Generate_Wang_Qi_Max_True_Data(N_x, N_u, S_test, m,theta_true);

    Input_Data["theta_true_ins=$ins"] = theta_true
    Input_Data["r_params_ins=$ins"] = r_params
    Input_Data["X_train_ins=$ins"] = X_train
    Input_Data["Y_train_ins=$ins"] = Y_train
    Input_Data["Z_train_ins=$ins"] = Z_train
    Input_Data["X_test_ins=$ins"] = X_test
    Input_Data["Y_test_ins=$ins"] = Y_test
    Input_Data["Z_test_ins=$ins"] = Z_test
    
    ins = ins + 1
    println(data_dir_ins)
end
save(string(data_dir,"Input_Data.jld2"),Input_Data)

/Users/zhangxun/Dropbox/Research/Robust_Exp/Data/Product_Line_Design/N=2_N_x=20_N_u=5_S_train=200_N_nonzero=5/instance=1/
/Users/zhangxun/Dropbox/Research/Robust_Exp/Data/Product_Line_Design/N=2_N_x=20_N_u=5_S_train=200_N_nonzero=5/instance=2/
/Users/zhangxun/Dropbox/Research/Robust_Exp/Data/Product_Line_Design/N=2_N_x=20_N_u=5_S_train=200_N_nonzero=5/instance=3/
/Users/zhangxun/Dropbox/Research/Robust_Exp/Data/Product_Line_Design/N=2_N_x=20_N_u=5_S_train=200_N_nonzero=5/instance=4/
/Users/zhangxun/Dropbox/Research/Robust_Exp/Data/Product_Line_Design/N=2_N_x=20_N_u=5_S_train=200_N_nonzero=5/instance=5/
/Users/zhangxun/Dropbox/Research/Robust_Exp/Data/Product_Line_Design/N=2_N_x=20_N_u=5_S_train=200_N_nonzero=5/instance=6/
/Users/zhangxun/Dropbox/Research/Robust_Exp/Data/Product_Line_Design/N=2_N_x=20_N_u=5_S_train=200_N_nonzero=5/instance=7/
/Users/zhangxun/Dropbox/Research/Robust_Exp/Data/Product_Line_Design/N=2_N_x=20_N_u=5_S_train=200_N_nonzero=5/instance=8/
/Users/zhangxun/Dropbox/

In [ ]:
using LinearAlgebra, Optim, Random

# ------------------------------------------------------------
# 1. 预计算所有扩展设计矩阵
# ------------------------------------------------------------
function precompute_extended_designs(X::Vector{Matrix{Float64}}, Z::Matrix{Float64})
    n = length(X)
    d = size(X[1], 2)
    p = size(Z, 2)
    total_dim = (d + 1) * (p + 1)
    X_tilde = Vector{Matrix{Float64}}(undef, n)

    # 预分配临时向量以避免重复分配
    x_tilde_temp = Vector{Float64}(undef, total_dim)

    for i in 1:n
        mi = size(X[i], 1)
        Xt = Matrix{Float64}(undef, mi, total_dim)

        @inbounds for j in 1:mi
            x_ij = @view X[i][j, :]
            z_i = @view Z[i, :]

            idx = 1
            x_tilde_temp[idx] = 1.0
            idx += 1

            @views x_tilde_temp[idx:idx+d-1] .= x_ij
            idx += d

            @inbounds for k in 1:p
                zk = z_i[k]
                x_tilde_temp[idx] = zk
                idx += 1
                @views x_tilde_temp[idx:idx+d-1] .= zk .* x_ij
                idx += d
            end

            @views Xt[j, :] .= x_tilde_temp
        end
        X_tilde[i] = Xt
    end
    return X_tilde
end

# ------------------------------------------------------------
# 2. 快速单样本负对数似然（使用预计算的 Xt_i）
# ------------------------------------------------------------
function neg_log_likelihood_single_fast(theta::AbstractVector, Xt_i::AbstractMatrix, y_i::Int)
    mi = size(Xt_i, 1)
    # 计算所有产品的效用：<θ, x̃_ij> for j=1..mi
    utilities = Xt_i * theta          # mi 维向量
    # 添加默认选项（效用为 0）
    m_val = maximum(utilities)
    # 数值稳定的 log-sum-exp: log(sum(exp(u))) = m + log(sum(exp(u - m)))
    shifted_utils = utilities .- m_val
    sum_exp = sum(exp, shifted_utils) + exp(-m_val)  # + exp(0 - m_val) for default option
    log_denominator = m_val + log(sum_exp)

    if y_i == 0
        log_prob = -log_denominator  # 因为 U0 = 0 → log(exp(0)/denom) = -log_denom
    else
        log_prob = utilities[y_i] - log_denominator
    end

    return -log_prob
end

# ------------------------------------------------------------
# 3. 快速整体负对数似然
# ------------------------------------------------------------
function neg_log_likelihood_fast(theta::AbstractVector, X_tilde::Vector{Matrix{Float64}}, Y::Vector{Int})
    n = length(Y)
    total_neg_ll = 0.0
    @inbounds for i in 1:n
        total_neg_ll += neg_log_likelihood_single_fast(theta, X_tilde[i], Y[i])
    end
    return total_neg_ll / n
end

# ------------------------------------------------------------
# 4. Lasso 目标函数（使用预计算数据）
# ------------------------------------------------------------
function lasso_objective_fast(theta::AbstractVector, lambda::Float64, X_tilde::Vector{Matrix{Float64}}, Y::Vector{Int})
    nll = neg_log_likelihood_fast(theta, X_tilde, Y)
    l1_penalty = lambda * norm(theta, 1)
    return nll + l1_penalty
end

# ------------------------------------------------------------
# 5. 参数解析（不变）
# ------------------------------------------------------------
function parameter_divide(theta_hat, d::Int, p::Int)
    alpha0_hat = 0.0
    alpha_hat = zeros(d)
    beta_hat = zeros(p)
    A_hat = zeros(d, p)

    idx = 1
    alpha0_hat = theta_hat[idx]
    idx += 1

    alpha_hat = theta_hat[idx:idx+d-1]
    idx += d

    for k in 1:p
        beta_hat[k] = theta_hat[idx]
        idx += 1
        A_hat[:, k] = theta_hat[idx:idx+d-1]
        idx += d
    end
    return alpha0_hat, alpha_hat, beta_hat, A_hat
end

# ------------------------------------------------------------
# 6. 主估计函数（优化版）
# ------------------------------------------------------------
function estimate_parameters_fast(
    X::Vector{Matrix{Float64}},
    Y::Vector{Int64},
    Z::Matrix{Float64},
    lambda::Float64,
    d::Int,
    p::Int;
    initial_theta = nothing,
    precomputed_X_tilde = nothing
)
    # 预计算扩展设计（如果未提供）
    if isnothing(precomputed_X_tilde)
        @time X_tilde = precompute_extended_designs(X, Z)
    else
        X_tilde = precomputed_X_tilde
    end

    total_dim = (d + 1) * (p + 1)
    if isnothing(initial_theta)
        initial_theta = randn(total_dim) * 0.1
    end

    # 目标函数闭包
    obj(theta) = lasso_objective_fast(theta, lambda, X_tilde, Y)

    # 使用 OWLQN 优化（专为 L1 设计）
    result = optimize(
        obj,
        initial_theta,
        OWLQN(lambda),
        Optim.Options(show_trace = false, g_tol = 1e-6, iterations = 1000, time_limit = 300)
    )

    theta_hat = Optim.minimizer(result)
    alpha0_hat, alpha_hat, beta_hat, A_hat = parameter_divide(theta_hat, d, p)

    return alpha0_hat, alpha_hat, beta_hat, A_hat, result, X_tilde
end

In [ ]:
ins = 1
theta_true = Input_Data["theta_true_ins=$ins"]
r_params = Input_Data["r_params_ins=$ins"]
X_train = Input_Data["X_train_ins=$ins"]
Y_train = Input_Data["Y_train_ins=$ins"]
Z_train = Input_Data["Z_train_ins=$ins"]
X_test = Input_Data["X_test_ins=$ins"]
Y_test = Input_Data["Y_test_ins=$ins"]
Z_test = Input_Data["Z_test_ins=$ins"]

In [ ]:
lambda = 0.01

alpha0, alpha, beta, A, result, X_tilde = estimate_parameters_fast(X_train, Y_train, Z_train, lambda, N_x, N_u)

println("Converged: ", Optim.converged(result))
println("Objective: ", Optim.minimum(result))

In [12]:
using Pkg
Pkg.rm("Optim")
Pkg.gc()  # 清理缓存
Pkg.add("Optim")  # 不指定版本！

    Updating `~/Dropbox/Research/Robust_Exp/Code_Macbook/Robust_Exp/Project.toml`
  [429524aa] - Optim v1.13.2
    Updating `~/Dropbox/Research/Robust_Exp/Code_Macbook/Robust_Exp/Manifest.toml`
  [47edcb42] - ADTypes v1.18.0
  [79e6a3ab] - Adapt v4.4.0
  [4fba245c] - ArrayInterface v7.20.0
  [187b0558] - ConstructionBase v1.6.0
  [a0c0ee7d] - DifferentiationInterface v0.7.8
  [4e289a0a] - EnumX v1.0.5
  [6a86dc24] - FiniteDiff v2.28.1
  [d3d80556] - LineSearches v7.4.0
  [d41bc354] - NLSolversBase v7.10.0
  [429524aa] - Optim v1.13.2
  [d96e819e] - Parameters v0.12.3
  [85a6dd25] - PositiveFactorizations v0.2.4
  [efcf1570] - Setfield v1.1.2
  [3a884ed6] - UnPack v1.0.2
  [8ba89e20] - Distributed
  [9fa8497b] - Future
      Active manifest files: 8 found
      Active artifact files: 152 found
      Active scratchspaces: 13 found
     Deleted no artifacts, repos, packages or scratchspaces
   Resolving package versions...
    Updating `~/Dropbox/Research/Robust_Exp/Code_Macbook/Robust_Ex

In [13]:
using Pkg
Pkg.status("Optim")

Status `~/Dropbox/Research/Robust_Exp/Code_Macbook/Robust_Exp/Project.toml`
  [429524aa] Optim v1.13.2


In [10]:
using Optim: OWLQN   # 👈 关键！显式导入 OWLQN

UndefVarError: UndefVarError: `OWLQN` not defined